## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-29-16-55-00 +0000,wsj,Boeing Books Nearly $5 Billion Charge for 777X...,https://www.wsj.com/business/airlines/boeing-b...
1,2025-10-29-16-54-00 +0000,wsj,General Motors Lays Off Thousands of Electric-...,https://www.wsj.com/business/autos/general-mot...
2,2025-10-29-16-52-55 +0000,nyt,France Adds Consent to Its Rape Law After Year...,https://www.nytimes.com/2025/10/29/world/europ...
3,2025-10-29-16-51-28 +0000,nyt,Live Updates: Hurricane Melissa Turns Toward t...,https://www.nytimes.com/live/2025/10/29/weathe...
4,2025-10-29-16-51-10 +0000,nyt,"Halloween Adventure, the Store Where It Is Alw...",https://www.nytimes.com/2025/10/29/nyregion/ha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,58
49,new,20
85,korea,16
24,hurricane,15
25,melissa,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2025-10-29-00-39-18 +0000,cbc,North Korea says it test-fired cruise-missiles...,https://www.cbc.ca/news/world/north-korea-miss...,128
244,2025-10-29-02-10-44 +0000,nypost,North Korea boasts missile test-firings as Tru...,https://nypost.com/2025/10/28/world-news/north...,126
324,2025-10-28-20-45-01 +0000,latimes,News Analysis: Trade deal or trade truce? Ques...,https://www.latimes.com/politics/story/2025-10...,126
53,2025-10-29-15-17-29 +0000,nyt,"Feted as a Peacemaker in South Korea, Trump Vo...",https://www.nytimes.com/2025/10/29/world/asia/...,118
109,2025-10-29-13-04-18 +0000,wapo,"In South Korea, Trump secures a trade deal — a...",https://www.washingtonpost.com/politics/2025/1...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,128,2025-10-29-00-39-18 +0000,cbc,North Korea says it test-fired cruise-missiles...,https://www.cbc.ca/news/world/north-korea-miss...
324,68,2025-10-28-20-45-01 +0000,latimes,News Analysis: Trade deal or trade truce? Ques...,https://www.latimes.com/politics/story/2025-10...
169,64,2025-10-29-10-02-11 +0000,nypost,JD Vance tells ‘Pod Force One’ he’s ‘turned ov...,https://nypost.com/2025/10/29/us-news/jd-vance...
213,58,2025-10-29-08-02-27 +0000,nypost,Hurricane Melissa makes landfall in eastern Cu...,https://nypost.com/2025/10/29/world-news/hurri...
31,57,2025-10-29-16-04-58 +0000,bbc,"Israeli strikes in Gaza kill 104, health minis...",https://www.bbc.com/news/articles/cgjdy5eevn2o...
47,54,2025-10-29-15-31-44 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
361,47,2025-10-28-18-19-57 +0000,nypost,Foreign betting markets could influence NYC el...,https://nypost.com/2025/10/28/business/pro-mam...
57,43,2025-10-29-15-08-02 +0000,nypost,Nvidia becomes first $5T company powered by AI...,https://nypost.com/2025/10/29/business/nvidia-...
157,41,2025-10-29-10-58-08 +0000,nypost,Chicago lawyer charged in death of wife year a...,https://nypost.com/2025/10/29/us-news/chicago-...
254,37,2025-10-29-01-00-00 +0000,wsj,"From Amazon to Target, the country’s largest e...",https://www.wsj.com/economy/jobs/white-collar-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
